In [1]:
import glob
import netCDF4 as nc
import numpy as np
from osgeo import gdal,osr,ogr

In [2]:
Input_folder = r'.\nc'
Output_folder = r'.\tif\output'
# 读取所有nc数据
data_list = glob.glob(Input_folder + '/*.nc4')
data_list[:5]

['.\\nc\\GLDAS_NOAH025_M.A202101.021.nc4',
 '.\\nc\\GLDAS_NOAH025_M.A202102.021.nc4',
 '.\\nc\\GLDAS_NOAH025_M.A202103.021.nc4',
 '.\\nc\\GLDAS_NOAH025_M.A202104.021.nc4',
 '.\\nc\\GLDAS_NOAH025_M.A202105.021.nc4']

In [3]:
nc_data_obj = nc.Dataset(data_list[0])
print(nc_data_obj, type(nc_data_obj))  # 了解NC_DS的数据类型，<class 'netCDF4._netCDF4.Dataset'>
# print(nc_data_obj.variables)  # 了解变量的基本信息

<class 'netCDF4._netCDF4.Dataset'>
root group (NETCDF4 data model, file format HDF5):
    missing_value: -9999.0
    tavg definition:: past 3-hour average
    acc definition:: past 3-hour accumulation
    inst definition:: instantaneous
    title: GLDAS2.1 LIS land surface model output monthly mean
    institution: NASA GSFC
    source: Noah_v3.6 forced with GDAS-AGRMET-GPCPv13rA1_202101
    history: created on date: 2021-04-23T11:19:32.357
    references: Rodell_etal_BAMS_2004, Kumar_etal_EMS_2006, Peters-Lidard_etal_ISSE_2007
    conventions: CF-1.6
    comment: website: https://ldas.gsfc.nasa.gov/gldas, https://lis.gsfc.nasa.gov/
    MAP_PROJECTION: EQUIDISTANT CYLINDRICAL
    SOUTH_WEST_CORNER_LAT: -59.875
    SOUTH_WEST_CORNER_LON: -179.875
    DX: 0.25
    DY: 0.25
    dimensions(sizes): lon(1440), lat(600), time(1), bnds(2)
    variables(dimensions): float32 lat(lat), float32 lon(lon), float64 time(time), float64 time_bnds(time, bnds), float32 Swnet_tavg(time, lat, lon), float32

In [4]:
def NC_to_tiffs(data, Output_folder, bandname):
    nc_data_obj = nc.Dataset(data)
#     print(nc_data_obj, type(nc_data_obj))  # 了解NC_DS的数据类型，<class 'netCDF4._netCDF4.Dataset'>
#     print(nc_data_obj.variables)  # 了解变量的基本信息
#     print(nc_data_obj)
    Lon = nc_data_obj.variables['lon'][:]
    Lat = nc_data_obj.variables['lat'][:]

    u_arr = np.asarray(nc_data_obj.variables[bandname])  # 这里根据需求输入想要转换的波段名称
    # 影像的左上角和右下角坐标
    LonMin, LatMax, LonMax, LatMin = [Lon.min(), Lat.max(), Lon.max(), Lat.min()]
    # 分辨率计算
    N_Lat = len(Lat)
    N_Lon = len(Lon)
    Lon_Res = (LonMax - LonMin) / (float(N_Lon) - 1)
    Lat_Res = (LatMax - LatMin) / (float(N_Lat) - 1)
    for i in range(len(u_arr[:])):
        # 创建.tif文件
        driver = gdal.GetDriverByName('GTiff')
        out_tif_name = Output_folder + '/' + bandname +'_'  + data.split('\\')[2].split('.')[1][1:] +  '.tif'
        out_tif = driver.Create(out_tif_name, N_Lon, N_Lat, 1, gdal.GDT_Float32)
        # 设置影像的显示范围
        # -Lat_Res一定要是-的
        geotransform = (LonMin, Lon_Res, 0, LatMax, 0, -Lat_Res)
        out_tif.SetGeoTransform(geotransform)
        # 获取地理坐标系统信息，用于选取需要的地理坐标系统
        srs = osr.SpatialReference()
        srs.ImportFromEPSG(4326)  # 定义输出的坐标系为"WGS 84"，AUTHORITY["EPSG","4326"]
        out_tif.SetProjection(srs.ExportToWkt())  # 给新建图层赋予投影信息

        # 去除异常值
        u_arr[u_arr[:, :] == -9999.0] = np.nan
        # 数据写出
        out_tif.GetRasterBand(1).WriteArray(u_arr[i][::-1]*60*60*24*30)
        # 将数据写入内存，此时没有写入硬盘 此处[::-1]用于图像的垂直镜像对称，避免图像颠倒
        out_tif.FlushCache()  # 将数据写入硬盘
        del out_tif  # 注意必须关闭tif文件

In [5]:
for i in range(len(data_list)):
    data = data_list[i]
    NC_to_tiffs(data, Output_folder,"Evap_tavg")

## Clip

In [6]:
Input_folder = './tif/output/'
Output_folder = './tif/clip/'
# 读取所有nc数据
data_list = glob.glob(Input_folder + '/*.tif')
data_list[:5]

['./tif/output\\Evap_tavg_202101.tif',
 './tif/output\\Evap_tavg_202102.tif',
 './tif/output\\Evap_tavg_202103.tif',
 './tif/output\\Evap_tavg_202104.tif',
 './tif/output\\Evap_tavg_202105.tif']

In [7]:
def clipTif(data):
    input_shape = r"shp/Uzb.shp" 
    output_raster= Output_folder + data.split('\\')[1].split('.')[0].split("_")[-1] +".tif"
    # tif输入路径，打开文件
    input_raster = data
    # 矢量文件路径，打开矢量文件
    input_raster=gdal.Open(input_raster)
    ds = gdal.Warp(output_raster, # 裁剪图像保存完整路径（包括文件名）
                   input_raster, # 待裁剪的影像
                    # warpMemoryLimit=500 内存大小M
                   format='GTiff', # 保存图像的格式
                   cutlineDSName = input_shape, # 矢量文件的完整路径
                   cropToCutline = True,
                   copyMetadata=True,
                   creationOptions=['COMPRESS=LZW',"TILED=True"],
                   cutlineWhere="FIELD = 'whatever'",
                   dstNodata=None)

In [8]:
for i in range(len(data_list)):
    data = data_list[i]
    clipTif(data)

## statistics

In [9]:
def readTif(fileName):
    dataset = gdal.Open(fileName)
    im_width = dataset.RasterXSize #栅格矩阵的列数
    im_height = dataset.RasterYSize #栅格矩阵的行数
#     print(im_width, im_height)
    im_data = dataset.ReadAsArray(0,0,im_width,im_height)#获取数据
    im =  im_data[0:im_height,0:im_width]#获取蓝波段
    return im

In [10]:
Input_folder = './tif/clip/'
Output_folder = './tif/sum/'
# 读取所有nc数据
data_list = glob.glob(Input_folder + '/*.tif')
data_list[:5]

['./tif/clip\\202101.tif',
 './tif/clip\\202102.tif',
 './tif/clip\\202103.tif',
 './tif/clip\\202104.tif',
 './tif/clip\\202105.tif']

In [11]:
dataset = gdal.Open(data_list[0])
im_width = dataset.RasterXSize #栅格矩阵的列数
im_height = dataset.RasterYSize #栅格矩阵的行数
im_bands = dataset.RasterCount #波段数
im_data = dataset.ReadAsArray(0,0,im_width,im_height)#获取数据
im_geotrans = dataset.GetGeoTransform()#获取仿射矩阵信息
im_proj = dataset.GetProjection()#获取投影信息

In [12]:
def writeTiff(im_data,path,im_width,im_height,im_bands,im_geotrans,im_proj):
    if 'int8' in im_data.dtype.name:
        datatype = gdal.GDT_Byte
    elif 'int16' in im_data.dtype.name:
        datatype = gdal.GDT_UInt16
    else:
        datatype = gdal.GDT_Float32

    if len(im_data.shape) == 3:
        im_bands, im_height, im_width = im_data.shape
    elif len(im_data.shape) == 2:
        im_data = np.array([im_data])
    else:
        im_bands, (im_height, im_width) = 1,im_data.shape
        #创建文件
    driver = gdal.GetDriverByName("GTiff")
    
    path = Output_folder + "\\" + path.split('\\')[1].split('.')[0][:4] + ".tif"
    dataset = driver.Create(path, im_width, im_height, im_bands, datatype)
    if(dataset!= None):
        dataset.SetGeoTransform(im_geotrans) #写入仿射变换参数
        dataset.SetProjection(im_proj) #写入投影
    for i in range(im_bands):
        dataset.GetRasterBand(i+1).WriteArray(im_data[i])
    del dataset

In [13]:
len(data_list)

12

In [14]:
index = 1
tmp = np.zeros(readTif(data_list[0]).shape ,dtype=float)
for i in range(0, len(data_list)):
    if index % 12 == 0:
        print(index / 12)
        writeTiff(tmp,data_list[i -1] ,im_width ,im_height ,im_bands ,im_geotrans ,im_proj)
        tmp = np.zeros(readTif(data_list[i]).shape ,dtype=float)

    tmp += np.array(readTif(data_list[i]))
    index+=1

1.0
